In [374]:
]box on
]rows on
⎕PW←1000

Was ON

Was ON

## Part one

First, let's figure out how to map letters to priorities. This magic incantation gives us an array of the lowercase letters followed by the uppercase letters. Lucky for us, their indices (1-indexed by default in APL) is their priority.

In [12]:
⊢letters←(⎕UCS 96+⍳26),⎕A

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ

With that, we can use "index of" (dyadic ⍳) to get the priorities of each letter:

In [375]:
letters⍳'czba'

3 26 2 1

Ok, let's shelve that for a minute and figure out how to find the duplicated items in each rucksack.

First, we need to divide each line in two. For that, we can use "reshape" (dyadic ⍴). It takes an array of dimensions on the left and an array to reshape on the right, and returns a matrix with those dimensions:

In [377]:
2 3⍴1 2 3 4 5 6

1 2 3
4 5 6

Since ⍴ reads items out of the input array by filling up each row before moving onto the next one, to end up with half of each line in a row we want matrices with 2 rows and n/2 columns, where n is the length of the string.

Let's try it on the first line of input (⊃ monadically is "first"):

In [385]:
line←⊃input
2 (2÷⍨≢line)⍴line

vJrwpWtwJgWr
hcsFMMfFFhFp

There's a little going on in there, so let's break it down. ≢ (tally) returns the length of its input. To divide it by 2, we could write:
```
(≢line)÷2
```
But that's kind of awkward with APL's right-to-left evaluation semantics. Instead, we can use the "swap" (⍨) to flip the arguments of ÷, removing some pesky parentheses.

Next, we want to find the letter in common between those two rows. "Intersection" (∩) is perfect for that, when used dyadically it returns only the elements in common between its two arguments:

In [387]:
1 2 3∩3 4 5

3

But it wants its arguments to be arrays, and instead of two arrays, we currently have a single matrix. Luckily, "split" (↓) does exactly that: it turns matrices into nested arrays of rows:

In [389]:
2 3⍴1 2 3 4 5 6
↓2 3⍴1 2 3 4 5 6

1 2 3
4 5 6

┌─────┬─────┐
│1 2 3│4 5 6│
└─────┴─────┘

Now we can just "reduce" (/) each pair of array using ∩! Reduce is an operator that takes a array of arrays on the right and inserts its left-hand argument between each. For example, `+/1 2 3 4` is the same as `1 + 2 + 3 + 4`.

In [386]:
∩/↓2 (2÷⍨≢line)⍴line

┌─┐
│p│
└─┘

We might think we're done here, but if we map the code we just wrote over the entire test input, we get a different story:

In [424]:
{∩/↓2 (2÷⍨≢⍵)⍴⍵}¨input

┌───┬────┬────┬────┬─────┬─────┐
│┌─┐│┌──┐│┌──┐│┌──┐│┌───┐│┌───┐│
││p│││LL│││PP│││vv│││ttt│││sss││
│└─┘│└──┘│└──┘│└──┘│└───┘│└───┘│
└───┴────┴────┴────┴─────┴─────┘

First, what's with those curly braces and ⍵s? That's how we define a function in APL, called a dfn. The stuff in curly braces calculates the return value of the dfn, and ⍵ (omega) is its right-hand argument.

We're not really doing anything special with the dfn here, it's really just a matter of convenience. The reason it's useful here is twofold. First, in `2 (2÷⍨≢⍵)⍴⍵`, we need that line in two different places: on the right side of ⍴ (reshape) and to get the length of half the string. Second, we have other operations after ⍴ that we want applied once per line. It's convenient to just stuff them in the dfn and use that entire function as the right argument to each (¨), which is basically the "map" function you'd find in functional programming languages.

Ok, so what's with those repeated letters? Playing around with ∩ reveals the answer:

In [399]:
1 2 1 1 ∩ 1 1 0 0 2 2

1 2 1 1

Ok, so when the items in common between the two arguments appear more than once on the left, the return value contains *all* of those items in the order the appeared in the left-hand argument.

Well, that's fine for us. Since we know there will only be one letter in common between each half of the string, and we just need that letter once, we can use "unique" (∪) to remove the repeated values:

In [401]:
{∪¨∩/↓2 (2÷⍨≢⍵)⍴⍵}¨input

┌───┬───┬───┬───┬───┬───┐
│┌─┐│┌─┐│┌─┐│┌─┐│┌─┐│┌─┐│
││p│││L│││P│││v│││t│││s││
│└─┘│└─┘│└─┘│└─┘│└─┘│└─┘│
└───┴───┴───┴───┴───┴───┘

Great! Now we just need to turn those letters into numbers and add them up, and we're done with part 1.

First, let's collapse that nested array of 1-item arrays into a flat array using enlist (∊). No matter how nested a array is, ∊ just flattens it down to a array of values.

In [402]:
∊{∪¨∩/↓2 (2÷⍨≢⍵)⍴⍵}¨input

pLPvts

Now we can use the method we figured out earlier to turn them into numbers, and plus-reduce to sum the whole thing:

In [404]:
+/letters⍳∊{∪¨∩/↓2 (2÷⍨≢⍵)⍴⍵}¨input

157

And that's part one complete!

## Part two

For part two, we're doing almost the exact same thing, but instead of splitting each line in two, we're grouping lines into 3s and finding the common item. Reshape will work just fine for that though:

In [426]:
2 3⍴input

┌──────────────────────────────┬────────────────────────────────┬────────────────────────┐
│vJrwpWtwJgWrhcsFMMfFFhFp │jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL│PmmdzqPrVvPwwTWBwg │
├──────────────────────────────┼────────────────────────────────┼────────────────────────┤
│wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn│ttgJtRGJQctTZtZT │CrZsJsPPZsGzwwsLwLmpwMDw│
└──────────────────────────────┴────────────────────────────────┴────────────────────────┘

Of course, we want the number of rows to be dynamic, to be precise, the total length of the input divided by 3:

In [406]:
{(3÷⍨≢input) 3⍴⍵}input

┌──────────────────────────────┬────────────────────────────────┬────────────────────────┐
│vJrwpWtwJgWrhcsFMMfFFhFp │jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL│PmmdzqPrVvPwwTWBwg │
├──────────────────────────────┼────────────────────────────────┼────────────────────────┤
│wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn│ttgJtRGJQctTZtZT │CrZsJsPPZsGzwwsLwLmpwMDw│
└──────────────────────────────┴────────────────────────────────┴────────────────────────┘

Then find the intersections:

In [412]:
∩/{(3÷⍨≢input) 3⍴⍵}input

┌──┬─┐
│rr│Z│
└──┴─┘

Wait, did we just reduce over a matrix? I said reduce wants an array!

Well, when applied to matricies, reduce applies its operator between each row. Like many functions and operators in APL, it's *rank-polymorphic*. There's also a corresponding "reduce-first" (⌿) operator that reduces over columns instead of rows:

In [416]:
3 2⍴1 2 3 4 5 6
+/3 2⍴1 2 3 4 5 6
+⌿3 2⍴1 2 3 4 5 6

1 2
3 4
5 6

3 7 11

9 12

Returning to our problem, now we just need to unique the vectors and flatten,

In [421]:
∊∪¨∩/{(3÷⍨≢input) 3⍴⍵}input

rZ

Then convert them to numbers, and sum:

In [422]:
+/letters⍳∊∪¨∩/{(3÷⍨≢input) 3⍴⍵}input

70

And that's part two!

## My work in progress below, please ignore

In [14]:
↑⊢input←'vJrwpWtwJgWrhcsFMMfFFhFp' 'jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL' 'PmmdzqPrVvPwwTWBwg' 'wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn' 'ttgJtRGJQctTZtZT' 'CrZsJsPPZsGzwwsLwLmpwMDw'

vJrwpWtwJgWrhcsFMMfFFhFp 
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg 
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn 
ttgJtRGJQctTZtZT 
CrZsJsPPZsGzwwsLwLmpwMDw

In [35]:
(({((⍵-1)⍴1),0,⍵⍴1}2÷⍨≢)⊂⊢)'vJrwpWtwJgWrhcsFMMfFFhFp'

┌─┬─┬─┬─┬─┬─┬─┬─┬─┬─┬──┬─┬─┬─┬─┬─┬─┬─┬─┬─┬─┬─┬─┐
│v│J│r│w│p│W│t│w│J│g│Wr│h│c│s│F│M│M│f│F│F│h│F│p│
└─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴──┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┘

In [78]:
⍝ The take/drop approach is probably simpler than partition
12(⊂∘↑,⊂∘↓)⍳24

┌┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬─┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬──┬┬┬┬┬┬┬┬┬┬┬┬────────────────────────────────────────────────────────────────┐
││││││││││││1││││││││││││2││││││││││││3││││││││││││4││││││││││││5││││││││││││6││││││││││││7││││││││││││8││││││││││││9││││││││││││10││││││││││││11││││││││││││12││││││││││││13││││││││││││14││││││││││││15││││││││││││16││││││││││││17││││││││││││18││││││││││││19││││││││││││20││││││││││││21││││││││││││22││││││││││││23││││││││││││24││││││││││││┌──────────────────────────────────────────────────────────────┐│
││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ │││││││││││││1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24││
││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││ ││││││││││││└──────────────────────────────────────────────────────────────┘│
└┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴─┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴──┴┴┴┴┴┴┴┴┴┴┴┴────────────────────────────────────────────────────────────────┘

In [76]:
⍝ Want to use ⊂ monadically
↑ 12{(⊂⍺↑⍵),⊂⍺↓⍵}¨input

┌────────────┬────────────────────┐
│vJrwpWtwJgWr│hcsFMMfFFhFp │
├────────────┼────────────────────┤
│jqHRNqRjqzjG│DLGLrsFMfFZSrLrFZsSL│
├────────────┼────────────────────┤
│PmmdzqPrVvPw│wTWBwg │
├────────────┼────────────────────┤
│wMqvLMZHhHMv│wLHjbvcjnnSBnvTQFn │
├────────────┼────────────────────┤
│ttgJtRGJQctT│ZtZT │
├────────────┼────────────────────┤
│CrZsJsPPZsGz│wwsLwLmpwMDw │
└────────────┴────────────────────┘

In [107]:
↑ ∊/↑12{(⊂⍺↑⍵),⊂⍺↓⍵}¨input

0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 0 0 1
1 0 0 1 1 0 0 1 0 1 0 1
1 1 0 0 1 0 0 0 0 0 1 1
0 0 0 1 0 1 0 0 0 1 0 0

In [215]:
↑ (2÷⍨≢¨input){(⊂⍺↑⍵),⊂⍺↓⍵}¨input

┌────────────────┬────────────────┐
│vJrwpWtwJgWr │hcsFMMfFFhFp │
├────────────────┼────────────────┤
│jqHRNqRjqzjGDLGL│rsFMfFZSrLrFZsSL│
├────────────────┼────────────────┤
│PmmdzqPrV │vPwwTWBwg │
├────────────────┼────────────────┤
│wMqvLMZHhHMvwLH │jbvcjnnSBnvTQFn │
├────────────────┼────────────────┤
│ttgJtRGJ │QctTZtZT │
├────────────────┼────────────────┤
│CrZsJsPPZsGz │wwsLwLmpwMDw │
└────────────────┴────────────────┘

In [183]:
0 0 0 0 1 0 0 0 0 0 0 0/'vJrwpWtwJgWr'

p

In [164]:
{(⊃∊/12{(⊂⍺↑⍵),⊂⍺↓⍵}⍵)/⍵}'vJrwpWtwJgWrhcsFMMfFFhFp'
⍝ note: might be able to use ⊢ to get rid of defn
⍝ TODO: get rid of the hardcoded 12

LENGTH ERROR
      {(⊃∊/12{(⊂⍺↑⍵),⊂⍺↓⍵}⍵)/⍵}'vJrwpWtwJgWrhcsFMMfFFhFp'
                            ∧


In [217]:
{⊃∊/12{(⊂⍺↑⍵),⊂⍺↓⍵}⍵}'vJrwpWtwJgWrhcsFMMfFFhFp'

0 0 0 0 1 0 0 0 0 0 0 0

In [165]:
{{⊃∊/12{(⊂⍺↑⍵),⊂⍺↓⍵}⍵}⍵)/⍵}'vJrwpWtwJgWrhcsFMMfFFhFp'

SYNTAX ERROR
      {{⊃∊/12{(⊂⍺↑⍵),⊂⍺↓⍵}⍵}⍵)/⍵}'vJrwpWtwJgWrhcsFMMfFFhFp'
       ∧


In [219]:
⍝ Maybe the APL way is to do it in whole?
((2÷⍨≢¨input)↑¨input),⍨⍪∊/↑(2÷⍨≢¨input){(⊂⍺↑⍵),⊂⍺↓⍵}¨input

┌───────────────────────────────┬────────────────┐
│0 0 0 0 1 0 0 0 0 0 0 0 │vJrwpWtwJgWr │
├───────────────────────────────┼────────────────┤
│0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1│jqHRNqRjqzjGDLGL│
├───────────────────────────────┼────────────────┤
│1 0 0 0 0 0 1 0 0 │PmmdzqPrV │
├───────────────────────────────┼────────────────┤
│0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 │wMqvLMZHhHMvwLH │
├───────────────────────────────┼────────────────┤
│1 1 0 0 1 0 0 0 │ttgJtRGJ │
├───────────────────────────────┼────────────────┤
│0 0 0 1 0 1 0 0 0 1 0 0 │CrZsJsPPZsGz │
└───────────────────────────────┴────────────────┘

In [236]:
part1←{+/letters⍳⊃¨(//((2÷⍨≢¨⍵)↑¨⍵),⍨⍪∊/↑(2÷⍨≢¨⍵){(⊂⍺↑⍵),⊂⍺↓⍵}¨⍵)}

In [239]:
part1⊃⎕NGET'input.txt'1

7967

A second try, now that I know about union and intersection

In [325]:
part1←{+/∊{letters⍳∊∪¨∩/↓2 (2÷⍨≢⍵)⍴⍵}¨⍵}
part1 ⊃⎕NGET'input.txt'1

7967

Part 2

In [364]:
+/letters⍳∊∪¨∩/2 3⍴ input

70

In [370]:
⍝ now need to get rid of hardcoded shape
+/letters⍳∊∪¨∩/(3÷⍨≢input) 3⍴ input

70

In [373]:
part2←{+/letters⍳∊∪¨∩/(3÷⍨≢⍵) 3⍴⍵}
part2 ⊃⎕NGET'input.txt'1

2716

Success!

In [268]:
⍝ Theirs
⍝ r ← 58|20+⎕UCS¨ ⊃⎕NGET'in/03.txt'1
⍝ +/∊{∪¨∩/↓2(2÷⍨≢⍵)⍴⍵}¨r
∊{∪¨∩/↓ 2 (2÷⍨≢⍵)⍴⍵} ⊃ ⊃⎕NGET'input.txt'1

Q